<h1>UK Road Safety Analysis</h1>

<p>This notebook analyzes road accident data in the UK. The goal is to identify trends and patterns to improve road safety.</p>

<h2>Data Overview</h2>
<ul>
  <li>Date: The date of the accident.</li>
  <li>Location: The location of the accident.</li>
  <li>Severity: The severity of the accident.</li>
</ul>

<h2>Analysis</h2>
<ol>
  <li>Data Cleaning</li>
  <li>Exploratory Data Analysis (EDA)</li>
  <li>Visualization</li>
  <li>Conclusion</li>
</ol>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt